# Synthetic Feeder, Chennai


In this tutorial we are going to create a synthetic distribution feeder model by leveraging OpenStreet buildings as well as road data in from Chennai, India.

1. [Generating power system loads from Geometry objects](#load_to_geometry)
2. [Using clustering algorithms to create distribution transformers to serve loads](#dist_trans)
3. [ Generating high tension acyclic undirected network to connect distribution transformers using road network from OpenStreet data](#primary_network)
4. [Generating HT lines from primary network](#ht_lines)
5. [Generating secondary network](#lt_lines)

Developed by: Kapil Duwadi (Kapil.Duwadi@nrel.gov), 2022-03-16


Let's get the styling out of the way first. I like to see the width little bit bigger than what is there by default.

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

<a name="load_to_geometry"></a>
### 1. Generating power system loads from Geometry objects
Let's get all the buildings from chennai India and print total number of builidings along with sample building.

In [38]:
from shift.geometry import BuildingsFromPlace
g = BuildingsFromPlace("Kathmandu, Nepal", max_dist=300)
geometries = g.get_geometries()
print(f"Total number of buildings: {len(geometries)}" +
f"\nSample geometry: {geometries[0]}")

Total number of buildings: 1116
Sample geometry: Building( Latitude = 27.7069613,  Longitude = 85.3226836, Area = 0)


In order to create power system loads from geometries we need to tell how to assign phases, voltages and connection type for loads. In this case let's use RandomPhaseAllocator class to allocate phases to all geometries. Here we are saying all geometries are of single phase type and there are no two phase and three phase loads and finally pass all the geometries.

Similarly we initialize simple voltage setter by passing line to line voltage of 13.2 kV. DefaultConnSetter class is created which will set the connection type to 'wye' for all geometries.

In [39]:
from shift.load_builder import (RandomPhaseAllocator, 
                                SimpleVoltageSetter, DefaultConnSetter)
rpa = RandomPhaseAllocator(100, 0, 0, geometries)
svs = SimpleVoltageSetter(13.2)
dcs = DefaultConnSetter()

But wait how do we get power consumption data for the load. In order get consumption let's use building area info to get the kw.
Let's say we know there is a piecewise linear function that relates building area with peak kW consumption.

In [40]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

area_to_kw_curve = [(0,5), (10, 5.0), (20, 18), (50, 30)]

df = pd.DataFrame({
    'Area in meter square' : [x[0] for x in area_to_kw_curve],
    'Peak consumption in kW' : [x[1] for x in area_to_kw_curve]
})
fig = px.line(df, x="Area in meter square", y="Peak consumption in kW") 
fig.show()

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In order to use this piecewise linear function we can invoke PiecewiseBuildingAreaToConsumptionConverter class from load_builder and pass it as an argument to build the load. Let's see this class in action.


In [41]:
from shift.load_builder import PiecewiseBuildingAreaToConsumptionConverter
pbacc = PiecewiseBuildingAreaToConsumptionConverter(area_to_kw_curve)
area = 15
print(f"For area of {area} m^2 the consumption would be {pbacc.convert(area)}")

For area of 15 m^2 the consumption would be 11.5


Let's build the loads from the geometries and print one them. Here let's try to build constant power factor of 1.0 type loads for simplicity.


In [42]:
from shift.load_builder import ConstantPowerFactorBuildingGeometryLoadBuilder
from shift.load_builder import LoadBuilderEngineer
loads = []
for g in geometries:
    builder = ConstantPowerFactorBuildingGeometryLoadBuilder(g, 
                            rpa, pbacc, svs, dcs, 1.0)
    b = LoadBuilderEngineer(builder)
    loads.append(b.get_load())
print(len(loads), loads[0], loads[1])

1116 ConstantPowerFactorLoad(Name = 85.31784056057502_27.708509533652407_load, Latitude = 27.708509533652407, Longitude = 85.31784056057502, Phase = Phase.AN NumPhase = NumPhase.SINGLE, Connection Type = LoadConnection.STAR, kw = 5.0, pf = 1.0, kv = 7.621) ConstantPowerFactorLoad(Name = 85.32263532197685_27.707751238458343_load, Latitude = 27.707751238458343, Longitude = 85.32263532197685, Phase = Phase.CN NumPhase = NumPhase.SINGLE, Connection Type = LoadConnection.STAR, kw = 5.0, pf = 1.0, kv = 7.621)


Let's visualize these loads on top of GIS map. That would be cool right ?. In order to do that let's invoke two layer distribution network first.

<script src="https://cdn.plot.ly/plotly-2.12.1.min.js"></script>

In [43]:
from shift.feeder_network import (SimpleTwoLayerDistributionNetworkBuilder, 
                                  TwoLayerNetworkBuilderDirector)
network_builder = SimpleTwoLayerDistributionNetworkBuilder()
network = TwoLayerNetworkBuilderDirector(loads, [], [], [], network_builder)

In [44]:
from shift.network_plots import PlotlyGISNetworkPlot
from shift.constants import PLOTLY_FORMAT_CUSTOMERS_ONLY

API_KEY = None
p = PlotlyGISNetworkPlot(
        network.get_network(),
        API_KEY,
        'carto-darkmatter',
        asset_specific_style=PLOTLY_FORMAT_CUSTOMERS_ONLY
    )
p.show()

<a name="dist_trans"></a>
### 2. Using clustering algorithms to create distribution transformers to serve loads
Next step is to use clustering algorithm to figure out best location for positioning distribution transformers. Kmeans clustering is one way of doing this. Let's say we want to have 20 disribution transformers for this feeder.

In [45]:
from shift.clustering import KmeansClustering
import numpy as np

x_array = np.array([[load.longitude, load.latitude] 
                    for load in loads])
# Try to use kW, along with location information for clustering 
cluster_ = KmeansClustering(2)
clusters = cluster_.get_clusters(x_array)
#print(f'Optimal number of clusters: {cluster_.optimal_clusters}')
#cluster_.plot_scores()
cluster_.plot_clusters()

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



Let's use clustering algorithm to position distribution transformers. In order to create transformers from clusters we need to provide list of loads, clustering object, kV and connection type for both high tension and low tension side of transformers along with number of phase to be used to design transformers. The diversity factor function, power factor and adjustment_factor is used to compute kVA capacity for the transformer. The catalog is used to find nearest transformer capacity. Let's print the catalog and transformers.

In [46]:
from shift.transformer_builder import (
    ClusteringBasedTransformerLoadMapper)
from shift.enums import (TransformerConnection, 
                         NumPhase, Phase)

# Planned years of operation
# Actual years of operation
# Assumed forecasted annual load growth
# Actual annual growth rate
# Adjustment factor

trans_builder = ClusteringBasedTransformerLoadMapper(
    loads,
    clustering_object = cluster_,
    diversity_factor_func = lambda x: 0.3908524*np.log(x) + 1.65180707,
    ht_kv = 13.2,
    lt_kv = 0.4,
    ht_conn = TransformerConnection.DELTA,
    lt_conn = TransformerConnection.STAR,
    ht_phase = Phase.ABC,
    lt_phase = Phase.ABCN,
    num_phase = NumPhase.THREE,
    power_factor=0.9,
    adjustment_factor=1.15
)
t  = trans_builder.get_transformer_load_mapping()

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.



Let's visualize the loads and transformers in GIS map.

In [47]:
from shift.constants import (
    PLOTLY_FORMAT_CUSTOMERS_AND_DIST_TRANSFORMERS_ONLY)
network = TwoLayerNetworkBuilderDirector(loads, 
            list(t.keys()), [], [], network_builder)
p = PlotlyGISNetworkPlot(
        network.get_network(),
        API_KEY,
        'carto-darkmatter',
        asset_specific_style=PLOTLY_FORMAT_CUSTOMERS_AND_DIST_TRANSFORMERS_ONLY
    )
p.show()

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



<a name="primary_network"></a>
### 3. Generating high tension acyclic undirected network to connect distribution transformers using road network from OpenStreet data
Let's try to get road network from chennai india. We will convert the road network to undirected graph and use minimum spanning tree to remove any loops. 

In [49]:
from shift.graph import RoadNetworkFromPlace
from shift.constants import PLOTLY_FORMAT_SIMPLE_NETWORK
graph = RoadNetworkFromPlace('Kathmandu, Nepal', max_dist=300)
graph.get_network()
p = PlotlyGISNetworkPlot(
            graph.updated_network,
            API_KEY,
            'carto-darkmatter',
            asset_specific_style=PLOTLY_FORMAT_SIMPLE_NETWORK
        )
p.show()

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



Let's create an instance of Primary Network Builder class and visulaize the generated network.

In [36]:
from shift.primary_network_builder import PrimaryNetworkFromRoad
pnet = PrimaryNetworkFromRoad(
        graph,
        t,
        (80.2786311, 13.091658),
        lambda x: 0.3908524*np.log(x) + 1.65180707,
        13.2,
        100
    )
pnet.update_network_with_ampacity()
p = PlotlyGISNetworkPlot(
            pnet.get_sliced_graph(),
            API_KEY,
            'carto-darkmatter',
            asset_specific_style=PLOTLY_FORMAT_SIMPLE_NETWORK
        )
p.show()
p = PlotlyGISNetworkPlot(
            pnet.get_primary_network(),
            API_KEY,
            'carto-darkmatter',
            asset_specific_style=PLOTLY_FORMAT_SIMPLE_NETWORK
        )
p.show()

<class 'networkx.utils.decorators.argmap'> compilation 36:4: FutureWarning:

laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



Let's try to plot ampacity of primary line section versus how far it is from the substation.

<a name="ht_lines"></a>
### 4. Generating HT lines from primary network



In [37]:
from shift.primary_network_builder import PrimarySectionsBuilder
from shift.enums import ConductorType, NumPhase
from shift.line_section import HorizontalThreePhaseConfiguration
from shift.feeder_network import update_transformer_locations
print(pnet.retain_nodes)
print(pnet.substation_node)
psections = PrimarySectionsBuilder(
    pnet.get_network(),
    ConductorType.OVERHEAD,
    {NumPhase.THREE: HorizontalThreePhaseConfiguration(9, 0.4, "m")},
    NumPhase.THREE,
    Phase.ABC,
    False,
    material="ACSR",
)
longest_length = pnet.get_longest_length_in_kvameter() / 1609.34
k_drop = 2 / (longest_length)
l_sections = psections.generate_primary_line_sections(k_drop, 11.0)
r_nodes = pnet.get_trans_node_mapping()
t = update_transformer_locations(r_nodes, t, l_sections)
print(l_sections[0])

{'80.2695028_13.0847626_node': {'centre': (80.2695028, 13.0847626), 'longitude': 80.2695028, 'latitude': 13.0847626}, '80.2689645_13.0819377_node': {'centre': (80.2689645, 13.0819377), 'longitude': 80.2689645, 'latitude': 13.0819377}, '80.2702344_13.0857201_node': {'centre': (80.2786311, 13.091658), 'longitude': 80.2702344, 'latitude': 13.0857201}}
80.2702344_13.0857201_node
GeometryBasedLine(Name = 80.2696283_13.0829503_node__80.2697735_13.08341_node, FromNode = 80.2696283_13.0829503_node, ToNode = 80.2697735_13.08341_node, Length = 53.2391782199997 NumPhase = NumPhase.THREE, Length unit = m,  Geometry = OverheadLineGeometry(Name = 475b3162-f00e-4652-be27-cbf953c2b104_linegeometry, NumPhase = NumPhase.THREE, Number of conductors = 3, Configuration = <shift.line_section.HorizontalThreePhaseConfiguration object at 0x000001DFDC312610>, Phase wire = Wire(Name = 6_6/1_ACSR, Resistance unit = mi, GMR unit = ft, Radius unit = in, AC resistance (ohm per) = 3.98, Diameter = 0.198, GMR AC = 0.0

In [15]:
from shift.constants import PLOTLY_FORMAT_CUSTOMERS_DIST_TRANSFORMERS_HT_LINE
network_builder = SimpleTwoLayerDistributionNetworkBuilder()
n = TwoLayerNetworkBuilderDirector(
    loads, 
    list(t.keys()),
    l_sections,
    [],
    network_builder)
p = PlotlyGISNetworkPlot(
    n.get_network(),
    API_KEY,
    'carto-darkmatter',
    asset_specific_style=PLOTLY_FORMAT_CUSTOMERS_DIST_TRANSFORMERS_HT_LINE
)
p.show()

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



<a name="lt_lines"></a>
### 5. Generating secondary network

In [20]:
from shift.secondary_network_builder import SecondaryNetworkBuilder, SecondarySectionsBuilder
from shift.line_section import HorizontalThreePhaseNeutralConfiguration, HorizontalSinglePhaseConfiguration
s_sections = []
counter = 0
load_to_node_mapping_dict = {}
for trans, cust_list in t.items():
    sn = SecondaryNetworkBuilder(cust_list, trans, lambda x: 0.3908524*np.log(x) + 1.65180707, 0.4, f"_secondary_{counter}")
    sn.update_network_with_ampacity()
    load_to_node_mapping_dict.update(sn.get_load_to_node_mapping())
    k_drop = 5 / (200 * 2)
    sc = SecondarySectionsBuilder(
        sn.get_network(),
        ConductorType.OVERHEAD,
        {
            NumPhase.THREE: HorizontalThreePhaseNeutralConfiguration(
                9, 0.4, 9.4, "m"
            )
        },
        {
            NumPhase.SINGLE: HorizontalSinglePhaseConfiguration(9, "m"),
        },
        NumPhase.THREE,
        Phase.ABCN,
        True,
        material="ACSR"
    )
    s_sections.extend(sc.generate_secondary_line_sections(k_drop, 0.4))
    counter +=1

Vertical distance 301.9700040697833m, Horizontal distance 597.3387336887437m
Vertical sections: 9,horizontal sections: 18
The node 80.27051087777778_13.084242662474946__secondary_0_node is not in the graph.
The node 80.27020487499999_13.084242662474946__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084242662474946__secondary_0_node is not in the graph.
The node 80.27020487499999_13.084242662474946__secondary_0_node is not in the graph.
The node 80.27020487499999_13.084545942375502__secondary_0_node is not in the graph.
The node 80.26989887222221_13.084545942375502__secondary_0_node is not in the graph.
The node 80.27051087777778_13.08393938257439__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084242662474946__secondary_0_node is not in the graph.
The node 80.27051087777778_13.08393938257439__secondary_0_node is not in the graph.
The node 80.27081688055556_13.08393938257439__secondary_0_node is not in the graph.
The node 80.26806285555556_13.0

The node 80.27020487499999_13.085152502176612__secondary_0_node is not in the graph.
The node 80.27051087777778_13.085152502176612__secondary_0_node is not in the graph.
The node 80.27020487499999_13.084545942375502__secondary_0_node is not in the graph.
The node 80.27020487499999_13.084849222276057__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084545942375502__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084849222276057__secondary_0_node is not in the graph.
The node 80.27020487499999_13.084545942375502__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084242662474946__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084545942375502__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084242662474946__secondary_0_node is not in the graph.
The node 80.27051087777778_13.084545942375502__secondary_0_node is not in the graph.
The node 80.27020487499999_13.084849222276057__secondary_0_node i

<class 'networkx.utils.decorators.argmap'> compilation 36:4: FutureWarning:

laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.



Vertical distance 293.1726471283571m, Horizontal distance 581.6018716309948m
Vertical sections: 9,horizontal sections: 18
The node 80.26761493790872_13.082410222222222__secondary_1_node is not in the graph.
The node 80.26761493790872_13.082704666666666__secondary_1_node is not in the graph.
The node 80.26791287581744_13.082410222222222__secondary_1_node is not in the graph.
The node 80.26791287581744_13.082704666666666__secondary_1_node is not in the graph.
The node 80.26731699999999_13.082410222222222__secondary_1_node is not in the graph.
The node 80.26761493790872_13.082410222222222__secondary_1_node is not in the graph.
The node 80.26731699999999_13.082115777777776__secondary_1_node is not in the graph.
The node 80.26761493790872_13.082115777777776__secondary_1_node is not in the graph.
The node 80.26731699999999_13.081821333333332__secondary_1_node is not in the graph.
The node 80.26761493790872_13.081821333333332__secondary_1_node is not in the graph.
The node 80.26731699999999_1

The node 80.27029637908723_13.082704666666666__secondary_1_node is not in the graph.
The node 80.27059431699595_13.082704666666666__secondary_1_node is not in the graph.
The node 80.27059431699595_13.082999111111112__secondary_1_node is not in the graph.
The node 80.27029637908723_13.082999111111112__secondary_1_node is not in the graph.
The node 80.27059431699595_13.082999111111112__secondary_1_node is not in the graph.
The node 80.26850875163488_13.081821333333332__secondary_1_node is not in the graph.
The node 80.26850875163488_13.082115777777776__secondary_1_node is not in the graph.
The node 80.26880668954361_13.081821333333332__secondary_1_node is not in the graph.
The node 80.26880668954361_13.082115777777776__secondary_1_node is not in the graph.
The node 80.26880668954361_13.081821333333332__secondary_1_node is not in the graph.
The node 80.26910462745234_13.081526888888886__secondary_1_node is not in the graph.
The node 80.26910462745234_13.081821333333332__secondary_1_node i

The node 80.27089225490467_13.082410222222222__secondary_1_node is not in the graph.
The node 80.27119019281339_13.082410222222222__secondary_1_node is not in the graph.
The node 80.27089225490467_13.082704666666666__secondary_1_node is not in the graph.
The node 80.27119019281339_13.082704666666666__secondary_1_node is not in the graph.
The node 80.27089225490467_13.082410222222222__secondary_1_node is not in the graph.
The node 80.27119019281339_13.082410222222222__secondary_1_node is not in the graph.
The node 80.27119019281339_13.082704666666666__secondary_1_node is not in the graph.
The node 80.27119019281339_13.082704666666666__secondary_1_node is not in the graph.
The node 80.27148813072212_13.082704666666666__secondary_1_node is not in the graph.
The node 80.27178606863085_13.082410222222222__secondary_1_node is not in the graph.
The node 80.27178606863085_13.082410222222222__secondary_1_node is not in the graph.
The node 80.27178606863085_13.082704666666666__secondary_1_node i

The node 80.27119019281339_13.080937999999996__secondary_1_node is not in the graph.
The node 80.27119019281339_13.081232444444442__secondary_1_node is not in the graph.
The node 80.27148813072212_13.080937999999996__secondary_1_node is not in the graph.
The node 80.27148813072212_13.081232444444442__secondary_1_node is not in the graph.
The node 80.27119019281339_13.081232444444442__secondary_1_node is not in the graph.
The node 80.27119019281339_13.081526888888886__secondary_1_node is not in the graph.
The node 80.27148813072212_13.081232444444442__secondary_1_node is not in the graph.
The node 80.27148813072212_13.081526888888886__secondary_1_node is not in the graph.
The node 80.27119019281339_13.081232444444442__secondary_1_node is not in the graph.
The node 80.27119019281339_13.081526888888886__secondary_1_node is not in the graph.
The node 80.27148813072212_13.081232444444442__secondary_1_node is not in the graph.
The node 80.27148813072212_13.081526888888886__secondary_1_node i

<class 'networkx.utils.decorators.argmap'> compilation 36:4: FutureWarning:

laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.



In [21]:
from shift.constants import PLOTLY_FORMAT_ALL_ASSETS
from shift.enums import NetworkAsset
PLOTLY_FORMAT_ALL_ASSETS['nodes'][NetworkAsset.DISTXFMR]['color'] = 'red'
network_builder = SimpleTwoLayerDistributionNetworkBuilder()
n = TwoLayerNetworkBuilderDirector(
    loads, 
    list(t.keys()),
    l_sections,
    s_sections,
    network_builder)
p = PlotlyGISNetworkPlot(
    n.get_network(),
    None,
    'carto-darkmatter',
    asset_specific_style=PLOTLY_FORMAT_ALL_ASSETS
)
p.show()

C:\Users\KDUWADI\Miniconda3\envs\shift\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [28]:
from shift.transformer_builder import SingleTransformerBuilder
s_node = pnet.substation_node
s_coords = pnet.substation_coords
sub_trans_builder = SingleTransformerBuilder(
    loads,
    s_coords[0],
    s_coords[1],
    diversity_factor_func=lambda x: 0.3908524 * np.log(x) + 1.65180707,
    ht_kv=33,
    lt_kv=11,
    ht_conn=TransformerConnection.DELTA,
    lt_conn=TransformerConnection.STAR,
    ht_phase=Phase.ABC,
    lt_phase=Phase.ABCN,
    num_phase=NumPhase.THREE,
    power_factor=0.9,
    adjustment_factor=1.15,
)
st = sub_trans_builder.get_transformer_load_mapping()

from shift.exporter.opendss import (
ConstantPowerFactorLoadWriter,
TwoWindingSimpleTransformerWriter,
GeometryBasedLineWriter,
OpenDSSExporter
)
lw = ConstantPowerFactorLoadWriter(
    loads, load_to_node_mapping_dict, "loads.dss"
)
tw = TwoWindingSimpleTransformerWriter(list(t.keys()), "dist_xfmrs.dss")
stw = TwoWindingSimpleTransformerWriter(list(st.keys()), "sub_trans.dss")
sw = GeometryBasedLineWriter(
    l_sections + s_sections,
    line_file_name="lines.dss",
    geometry_file_name="line_geometry.dss",
    wire_file_name="wiredata.dss",
    cable_file_name="cabledata.dss",
)


dw = OpenDSSExporter(
    [tw, stw, sw, lw],
    r"C:\Users\KDUWADI\Desktop\NREL_Projects\shift_output",
    "master.dss",
    "chennai",
    33.0,
    50,
    Phase.ABCN,
    NumPhase.THREE,
    f"{s_node.split('_')[0]}_{s_node.split('_')[1]}_htnode",
    [0.001, 0.001],
    [0.001, 0.001],
    [0.4, 11.0, 33.0],
)
dw.export()